In [79]:
%pip install pandas

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [80]:
import pandas as pd
import numpy as np

In [81]:
df=pd.read_csv(r'C:\padhma\GUVI_CLASS_AUG2025\Projects\Amazon-Sales\amazon_india_complete_2015_2025.csv')

In [82]:
pd.set_option('display.max_columns', 150)
pd.set_option('display.max_rows', 150)

In [83]:
df.shape

(1127609, 34)

In [84]:
df.head(100)


,transaction_id,order_date,customer_id,product_id,product_name,category,subcategory,brand,original_price_inr,discount_percent,discounted_price_inr,quantity,subtotal_inr,delivery_charges,final_amount_inr,customer_city,customer_state,customer_tier,customer_spending_tier,customer_age_group,payment_method,delivery_days,delivery_type,is_prime_member,is_festival_sale,festival_name,customer_rating,return_status,order_month,order_year,order_quarter,product_weight_kg,is_prime_eligible,product_rating
0,TXN_2023_00063013,2023-07-23,CUST_2023_00018393,PROD_000454,Vivo Y95 64GB Black,Electronics,Smartphones,Vivo,27340.84,21.57,21442.59,1,21442.59,0.0,21442.59,Kolkata,West Bengal,Metro,Standard,36-45,COD,3,Standard,False,False,NaN,4.0,Delivered,7,2023,3,0.20,True,3.5
1,TXN_2021_00064486,20-07-2021,CUST_2015_00002865,PROD_000579,Realme Realme 3 128GB Black,Electronics,Smartphones,Realme,32907.49,0.00,32907.49,3,98722.47,0.0,98722.47,Ludhiana,Punjab,Tier2,Standard,NaN,UPI,3,Standard,False,False,NaN,5/5,Delivered,7,2021,3,0.21,False,4.5
2,TXN_2017_00065617,2017-11-16,CUST_2016_00004057,PROD_000295,Vivo V7 32GB Blue,Electronics,Smartphones,Vivo,"47,052.18",21.91,36741.38,1,36741.38,0.0,36741.38,Bangalore,Karnataka,Metro,Premium,36-45,Debit Card,6,Standard,False,False,NaN,5.0,Delivered,11,2017,4,0.24,True,4.3
3,TXN_2020_00054393,2020-05-04,CUST_2020_00014574,PROD_001654,Alienware Pavilion 4GB RAM Silver,Electronics,Laptops,Alienware,238725.44,59.60,96456.27,1,96456.27,0.0,96456.27,Bangalore,Karnataka,Metro,Premium,46-55,UPI,3,Standard,False,True,Summer Sale,5.0 stars,Delivered,5,2020,2,1.85,Yes,3.6
4,TXN_2018_00071646,2018-10-09,CUST_2018_00006275,PROD_000095,Motorola Moto X Play 16GB White,Electronics,Smartphones,Motorola,25970.76,0.00,25970.76,1,25970.76,0.0,25970.76,Kochi,Kerala,Tier2,Standard,18-25,COD,4,Standard,False,False,NaN,4.0,Delivered,10,2018,4,0.16,False,3.7
5,TXN_2016_00020663,2016-06-22,CUST_2016_00006056,PROD_001729,Xiaomi Slate 8GB RAM Silver,Electronics,Tablets,Xiaomi,21541.01,59.78,8663.17,1,8663.17,NaN,8663.17,Mumbai,Maharashtra,Metro,Standard,26-35,COD,4,Standard,0,True,Back to School,3.5,Delivered,6,2016,2,0.33,True,4.5
6,TXN_2019_00091040,2019-10-22,CUST_2019_00028838,PROD_000178,Xiaomi Mi 5 64GB Black,Electronics,Smartphones,Xiaomi,39859.05,45.78,21610.12,1,21610.12,0.0,21610.12,Delhi,Delhi,Metro,Budget,36-45,COD,5,Standard,False,True,Diwali Sale,NaN,Delivered,10,2019,4,0.24,True,3.7
7,TXN_2020_00052917,2020-05-24,CUST_2020_00016130,PROD_000539,OnePlus OnePlus 7 128GB Blue,Electronics,Smartphones,OnePlus,105554.58,0.00,105554.58,1,105554.58,0.0,105554.58,Mumbai,Maharashtra,Metro,Standard,NaN,UPI,4,Standard,False,FALSE,NaN,3.5,Delivered,5,2020,2,0.21,True,3.7
8,TXN_2022_00023262,2022-03-10,CUST_2015_00001067,PROD_001043,Realme Realme GT 2 128GB Black,Electronics,Smartphones,Realme,26198.55,64.51,9298.46,2,18596.91,0.0,18596.91,Kolkata,West Bengal,Metro,Standard,NaN,UPI,3,Standard,False,TRUE,Holi Festival,3.5,Delivered,3,2022,1,0.18,True,3.2
9,TXN_2017_00049484,2017-09-05,CUST_2017_00021979,PROD_000247,Samsung Galaxy S8+ 32GB White,Electronics,Smartphones,Samsung,121800.87,0.00,121800.87,3,365402.61,0.0,365402.61,Mumbai,Maharashtra,Metro,Standard,46-55,COD,6,Standard,False,False,NaN,5.0,Delivered,9,2017,3,0.19,True,4.3


In [85]:
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 2000)
#df.dtypes
#print(df.dtypes.to_string())
print(df.columns.tolist())



['transaction_id', 'order_date', 'customer_id', 'product_id', 'product_name', 'category', 'subcategory', 'brand', 'original_price_inr', 'discount_percent', 'discounted_price_inr', 'quantity', 'subtotal_inr', 'delivery_charges', 'final_amount_inr', 'customer_city', 'customer_state', 'customer_tier', 'customer_spending_tier', 'customer_age_group', 'payment_method', 'delivery_days', 'delivery_type', 'is_prime_member', 'is_festival_sale', 'festival_name', 'customer_rating', 'return_status', 'order_month', 'order_year', 'order_quarter', 'product_weight_kg', 'is_prime_eligible', 'product_rating']


In [86]:
#2 Cleaning original_price_inr column and converting it to float
df['original_price_inr'] = df['original_price_inr'].str.replace('Rs ',"").str.replace('₹',"").str.replace(",","").str.replace("Price on Request","").astype('float')

In [87]:
#3 Cleaning customer_rating
#df.dtypes['customer_rating']
df['customer_rating'] = df['customer_rating'].str.replace(' stars',"")



def clean_rating(x):
    if pd.isna(x):
        return np.nan
    else :
        x = str(x).strip().lower()

        if '/' in x:
            try:
                num, denom = x.split('/')
                num = float(num)
                denom = float(denom)
            except:
                return num
        else:
            return float(x)



# Apply to your dataframe
df['customer_rating'] = df['customer_rating'].apply(clean_rating)
df['customer_rating'].astype('float')



0          4.0
1          NaN
2          5.0
3          5.0
4          4.0
          ... 
1127604    5.0
1127605    4.5
1127606    4.0
1127607    5.0
1127608    5.0
Name: customer_rating, Length: 1127609, dtype: float64

In [88]:
#4 City column cleaning
city_map = {
    "bangalore": "Bengaluru",
    "bengaluru": "Bengaluru",
    "banglore": "Bengaluru",
    "bengalore": "Bengaluru",
    
    "mumbai": "Mumbai",
    "bombay": "Mumbai",
    "bambai": "Mumbai",
    "mumba": "Mumbai",

    "delhi": "Delhi",
    "new delhi": "Delhi",
    "delhi ncr": "Delhi",

    "chenai": "Chennai",
    "madras": "Chennai",

    "calcutta": "Kolkata",
    
    # Add more known corrections here
}


def clean_city(x):
    if pd.isna(x):
        return np.nan
    
    x = str(x).strip().lower()

    # If multiple city names like "Mumbai/Bombay"
    if "/" in x:
        parts = [p.strip() for p in x.split("/")]
        
        # Normalize each part using city_map
        cleaned_parts = [city_map.get(p, p) for p in parts]
        
        # Use the first standardized name as final
        return cleaned_parts[0].title()

    # Single city → normalize using dictionary
    x = city_map.get(x, x)
    
    return x.title()
df['customer_city'] = df['customer_city'].apply(clean_city)


In [89]:
vc = df['customer_city'].value_counts().reset_index()
vc.columns = ['city', 'count']
print(vc.to_string())




             city   count
0          Mumbai  141644
1           Delhi  123469
2       Bengaluru  102523
3         Chennai   84753
4            Pune   67703
5         Kolkata   66867
6       Ahmedabad   50861
7       Hyderabad   44573
8          Jaipur   40453
9           Surat   40400
10         Nagpur   37211
11         Kanpur   34334
12        Lucknow   34077
13         Indore   33592
14     Coimbatore   26695
15          Kochi   24940
16  Visakhapatnam   22167
17          Patna   21339
18       Vadodara   21078
19    Bhubaneswar   20472
20     Chandigarh   19575
21       Ludhiana   18891
22     Saharanpur    6705
23         Meerut    6641
24       Bareilly    6590
25        Aligarh    6407
26      Allahabad    6030
27       Varanasi    5897
28      Gorakhpur    5867
29      Moradabad    5855


In [90]:
#5 Boolean columns to consistent True/False format.
def clean_bool(x):
    if pd.isna(x):
        return None  # keep NaN as missing
    x = str(x).strip().lower()
    
    true_vals = {"true", "1", "yes", "y", "t"}
    false_vals = {"false", "0", "no", "n", "f"}
    
    if x in true_vals:
        return True
    if x in false_vals:
        return False
    
    return None   # unexpected values → treat as missing
bool_cols = ["is_prime_member", "is_prime_eligible", "is_festival_sale"]

for col in bool_cols:
    df[col] = df[col].apply(clean_bool)
    df[col] = df[col].fillna(False)
    df[col] = df[col].astype('boolean')  # pandas nullable boolean dtype
    

In [91]:
df[["is_prime_member", "is_prime_eligible", "is_festival_sale"]].head()

,is_prime_member,is_prime_eligible,is_festival_sale
0,False,True,False
1,False,False,False
2,False,True,False
3,False,True,True
4,False,False,False


In [92]:
#6 Product categories cleaning 

category_map = {
    "electronics": "Electronics",
    "electronic": "Electronics",
    "electronics & accessories": "Electronics",
    "electronics accessories": "Electronics",
    "electronic accessories": "Electronics",
    "electronic & accessories": "Electronics",
    "electronicss": "Electronics",
}



def clean_category(x):
    if pd.isna(x):
        return np.nan
    
    x = str(x).lower().strip()

    # Replace separators (/ & -)
    #x = re.sub(r"[\/&\-]", " ", x)
    x=x.replace("/"," ").replace("&"," ").replace("-"," ")

    # Remove multiple spaces
    #x = re.sub(r"\s+", " ", x)
    x=x.split()
    x=" ".join(x)

    # Cleanup: remove trailing words like "and", "etc" if needed
    x = x.replace("and", "").strip()

    # If mapped in dictionary → use standardized value
    if x in category_map:
        return category_map[x]
    
    # Else return cleaned title-case text
    return x.title()

df['category'] = df['category'].apply(clean_category)


In [93]:
#df.columns.tolist()
df['delivery_days'].value_counts()

delivery_days
3           288488
1           210462
4           206438
5           136608
2           126280
6           102668
7            34116
-1            6836
1-2 days      4534
Same Day      4469
Express       2317
0             2218
15            2175
Name: count, dtype: int64

In [94]:
#7 delivery_days cleaning

def extract_numbers(text):
    text = text.strip()
    nums = []

    temp = ""
    for ch in text:
        if ch.isdigit():
            temp += ch
        else:
            if temp != "":
                nums.append(int(temp))
                temp = ""
    if temp != "":
        nums.append(int(temp))

    return nums

def clean_delivery_days(x):

    if pd.isna(x):
        return np.nan

    x = str(x).strip().lower()

    # 1️⃣ Handle Same Day deliveries
    if "same" in x:
        return 0

    # 2️⃣ Extract numbers using only string logic
    nums = extract_numbers(x)

    # Nothing found → invalid
    if len(nums) == 0:
        return np.nan

    # 3️⃣ Range handling (e.g., 1-2 days)
    if len(nums) >= 2:
        return (nums[0] + nums[1]) / 2

    # 4️⃣ Single number (e.g., "3 days")
    return nums[0]

df["delivery_days"] = df["delivery_days"].apply(clean_delivery_days)

df["delivery_days"] = df["delivery_days"].apply(
    lambda x: np.nan if pd.notna(x) and (x < 0 or x > 30) else x
)
df["delivery_days"].astype('float')


0          3.0
1          3.0
2          6.0
3          3.0
4          4.0
          ... 
1127604    4.0
1127605    1.0
1127606    1.0
1127607    4.0
1127608    2.0
Name: delivery_days, Length: 1127609, dtype: float64

In [95]:

#df[df.duplicated(keep=False)]
#df.duplicated().sum()
#df.columns.tolist()
duplicate_keys =['customer_id', 'product_id', 'order_date', 'final_amount_inr']
dupes = df[df.duplicated(duplicate_keys, keep=False)]





In [96]:
print(df.columns.tolist())
#print(dupes.head())


['transaction_id', 'order_date', 'customer_id', 'product_id', 'product_name', 'category', 'subcategory', 'brand', 'original_price_inr', 'discount_percent', 'discounted_price_inr', 'quantity', 'subtotal_inr', 'delivery_charges', 'final_amount_inr', 'customer_city', 'customer_state', 'customer_tier', 'customer_spending_tier', 'customer_age_group', 'payment_method', 'delivery_days', 'delivery_type', 'is_prime_member', 'is_festival_sale', 'festival_name', 'customer_rating', 'return_status', 'order_month', 'order_year', 'order_quarter', 'product_weight_kg', 'is_prime_eligible', 'product_rating']


In [97]:
def detect_transaction_duplicates(
    df,
    subset_cols=["customer_id", "product_id", "order_date", "final_amount_inr"],
    bulk_threshold=5
):
    """
    Detect and classify duplicate transactions.

    Parameters
    ----------
    df : pandas.DataFrame
        Input dataset.
    subset_cols : list
        Columns used to determine duplicates.
    bulk_threshold : int
        Minimum count above which duplicates are treated as genuine (bulk orders).

    Returns
    -------
    dupes : DataFrame
        All duplicate transactions with classification.
    summary : DataFrame
        Summary count of genuine vs error duplicates.
    """

    # Step 1: Count duplicates using groupby (fastest)
    grp = df.groupby(subset_cols).size().reset_index(name="count")

    # Step 2: Filter groups where duplicates exist
    duplicate_groups = grp[grp["count"] > 1]

    if duplicate_groups.empty:
        print("No duplicates found based on given columns.")
        return None, None

    # Step 3: Merge back to original to extract duplicate rows
    dupes = df.merge(duplicate_groups[subset_cols], on=subset_cols, how="inner")

    # Step 4: Classify duplicates
    dupes["duplicate_type"] = dupes.groupby(subset_cols)["final_amount_inr"].transform("size")
    dupes["duplicate_type"] = dupes["duplicate_type"].apply(
        lambda x: "Genuine Bulk Order" if x >= bulk_threshold else "Data Error Duplicate"
    )

    # Step 5: Summary
    summary = dupes["duplicate_type"].value_counts().reset_index()
    summary.columns = ["duplicate_type", "count"]

    return dupes, summary






In [98]:

dupes, summary = detect_transaction_duplicates(df)
#print(summary.to_string())
# Keep only the data-error duplicates
error_dupes = dupes[dupes["duplicate_type"] == "Data Error Duplicate"]

# Remove these from original df
df_cleaned = df.merge(
    error_dupes[["customer_id", "product_id", "order_date", "final_amount_inr"]],
    on=["customer_id", "product_id", "order_date", "final_amount_inr"],
    how="left",
    indicator=True
).query("_merge == 'left_only'").drop(columns="_merge")
df_cleaned.shape

(1116389, 34)

In [99]:
#9 outlier price correction
df_cleaned.columns.tolist()
df_cleaned["original_price_inr"].describe()

count    1.116389e+06
mean     8.225163e+04
std      4.039435e+05
min     -3.001027e+05
25%      2.881544e+04
50%      4.654833e+04
75%      9.420046e+04
max      3.337169e+07
Name: original_price_inr, dtype: float64

In [100]:
def fix_decimal_outliers(df, price_col="original_price_inr", group_col="product_id"):
    stats = df.groupby(group_col)[price_col].median().to_dict()

    def auto_fix(row):
        price = row[price_col]
        median = stats[row[group_col]]

        if 9*median < price < 20*median:
            return price / 10
        if 90*median < price < 200*median:
            return price / 100
        if 900*median < price < 2000*median:
            return price / 1000
        return price

    df["corrected_price"] = df.apply(auto_fix, axis=1)
    return df
df_cleaned = fix_decimal_outliers(df_cleaned)

In [101]:
#10 Payment Method cleaning


payment_map = {
    "upi": "UPI",
    "phonepe": "UPI",
    "google pay": "UPI",
    "credit card": "Card",
    "credit_card": "Card",
    "cc": "Card",
    "debit card": "Card",
    "cash on delivery": "Cash",
    "cod": "Cash",
    "wallet": "Cash",
    "c.o.d": "Cash",
    "bnpl": "Cash",
    "net banking": "Net Banking",
}



def clean_payment(x):
    if pd.isna(x):
        return np.nan
    
    x = str(x).lower().strip()

    #x=x.split()
    #x=" ".join(x)


    # If mapped in dictionary → use standardized value
    if x in payment_map:
        return payment_map[x]
    
    # Else return cleaned title-case text
    return x.title()




In [102]:
df_cleaned['payment_method'] = df_cleaned['payment_method'].apply(clean_payment)
df_cleaned.value_counts('payment_method')

payment_method
UPI            380484
Cash           362327
Card           309309
Net Banking     64269
Name: count, dtype: int64

In [103]:
df_cleaned.isna().sum() #missing values check
#df.duplicated().sum() # duplication check

transaction_id                 0
order_date                     0
customer_id                    0
product_id                     0
product_name                   0
category                       0
subcategory                    0
brand                          0
original_price_inr             0
discount_percent               0
discounted_price_inr           0
quantity                       0
subtotal_inr                   0
delivery_charges           89319
final_amount_inr               0
customer_city                  0
customer_state                 0
customer_tier                  0
customer_spending_tier         0
customer_age_group        133965
payment_method                 0
delivery_days               2295
delivery_type                  0
is_prime_member                0
is_festival_sale               0
festival_name             769902
customer_rating           369319
return_status                  0
order_month                    0
order_year                     0
order_quar

In [104]:
# Missing values handling for delivery_charges
df_cleaned['delivery_charges'].value_counts(dropna=False)
#df_cleaned.shape #(1116389, 35)
#df_cleaned.head(10)
#df_cleaned['delivery_charges']
#df_cleaned['delivery_charges'] = df_cleaned['delivery_charges'].fillna(0.0)

#df_cleaned.dtypes

delivery_charges
0.0     1027062
NaN       89319
40.0          8
Name: count, dtype: int64

In [105]:
# delivery_days missing values handling
#df_cleaned['delivery_days'].value_counts(dropna=False)
#df_cleaned.dtypes
#df_cleaned.describe()
# Ensure we operate on the cleaned dataframe and use numeric dtype
df_cleaned['delivery_days'] = pd.to_numeric(df_cleaned['delivery_days'], errors='coerce')
mean_delivery = df_cleaned['delivery_days'].mean()
df_cleaned['delivery_days'].fillna(mean_delivery, inplace=True)
df_cleaned['delivery_days'].value_counts(dropna=False)

C:\Users\harih\AppData\Local\Temp\ipykernel_32696\949999276.py:8: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_cleaned['delivery_days'].fillna(mean_delivery, inplace=True)


delivery_days
3.000000     285594
1.000000     215124
4.000000     204351
5.000000     135247
2.000000     125064
6.000000     101644
7.000000      33802
0.000000       6635
1.500000       4478
3.322155       2295
15.000000      2155
Name: count, dtype: int64

In [106]:
# missing values in customer_age_group
#df_cleaned.dtypes
#df_cleaned['customer_age_group'].value_counts(dropna=False)
#df.describe()

# Calculate the mode
mode_value = df_cleaned['customer_age_group'].mode()[0] # .mode() returns a Series, [0] gets the value

# Fill the missing values with the mode
df_cleaned['customer_age_group'] = df_cleaned['customer_age_group'].fillna(mode_value)

print(df_cleaned['customer_age_group'].value_counts(dropna=False))


customer_age_group
26-35    478494
18-25    313274
36-45    197094
46-55     97996
55+       29531
Name: count, dtype: int64


In [107]:
#df_cleaned.dtypes
#df_cleaned.isna().sum()
#df_cleaned['festival_name'].value_counts(dropna=False) # more than 50% missing value
df_cleaned['festival_name'] = df_cleaned['festival_name'].fillna('No Festival')
#df_cleaned['festival_name'] = df_cleaned['festival_name'].replace('NaN','Unknown')
#df['festival_name'] = df['festival_name'].replace([np.nan, 'nan'], 'No Festival')

In [108]:
#df_cleaned['customer_rating'].value_counts(dropna=False) # more than 50% missing value
#df_cleaned['customer_rating'].describe()
#customer_rating           369319
median_rating = df_cleaned['customer_rating'].median()

# Fill the missing values with the calculated median
df_cleaned['customer_rating'] = df_cleaned['customer_rating'].fillna(median_rating)

# Verify the changes
print(df_cleaned['customer_rating'].value_counts(dropna=False))

customer_rating
4.5    621339
5.0    191680
4.0    182619
3.5     78061
3.0     42690
Name: count, dtype: int64


In [109]:
def smart_date_parse(x):
    for fmt in ("%Y-%m-%d", "%d-%m-%Y", "%d/%m/%Y", "%m/%d/%Y", "%Y/%m/%d", "%d.%m.%Y"):
        try:
            return pd.to_datetime(x, format=fmt)
        except:
            pass
    return pd.NaT

df_cleaned['order_date'] = df_cleaned['order_date'].astype(str).apply(smart_date_parse)
df_cleaned['order_date'] = pd.to_datetime(df_cleaned['order_date'], dayfirst=True, errors='coerce')



In [118]:
df_cleaned.loc[df_cleaned['order_date'].isna(), 'order_date'].unique()[:20]


<DatetimeArray>
[]
Length: 0, dtype: datetime64[ns]

In [112]:
'''
df_cleaned['order_date'] = pd.to_datetime(
    df['order_date'],
    format='mixed',
    dayfirst=True,   # FIXES 20-07-2021
    errors='coerce'
)
'''


"\ndf_cleaned['order_date'] = pd.to_datetime(\n    df['order_date'],\n    format='mixed',\n    dayfirst=True,   # FIXES 20-07-2021\n    errors='coerce'\n)\n"

In [113]:
df_cleaned.to_csv(r'C:\padhma\GUVI_CLASS_AUG2025\Projects\Amazon-Sales\cleaned.csv', index=False)

In [117]:
print("Sample transactions:\n", df_cleaned.head())
print("Null counts:\n", df_cleaned.isna().sum())
print("Dtypes:\n", df_cleaned.dtypes)

Sample transactions:
       transaction_id order_date         customer_id   product_id                       product_name     category  subcategory      brand  original_price_inr  discount_percent  discounted_price_inr  quantity  subtotal_inr  delivery_charges  final_amount_inr customer_city customer_state customer_tier customer_spending_tier customer_age_group payment_method  delivery_days delivery_type  is_prime_member  is_festival_sale festival_name  customer_rating return_status  order_month  order_year  order_quarter  product_weight_kg  is_prime_eligible  product_rating  corrected_price
0  TXN_2023_00063013 2023-07-23  CUST_2023_00018393  PROD_000454                Vivo Y95 64GB Black  Electronics  Smartphones       Vivo            27340.84             21.57              21442.59         1      21442.59               0.0          21442.59       Kolkata    West Bengal         Metro               Standard              36-45           Cash            3.0      Standard            Fals

In [115]:
print(df_cleaned['order_date'].head(20))

0    2023-07-23
1    2021-07-20
2    2017-11-16
3    2020-05-04
4    2018-10-09
5    2016-06-22
6    2019-10-22
7    2020-05-24
8    2022-03-10
9    2017-09-05
10   2018-12-19
11   2019-01-07
12   2020-12-25
13   2017-11-24
14   2018-09-22
15   2015-10-08
16   2022-02-27
17   2025-09-01
18   2017-06-20
19   2020-05-17
Name: order_date, dtype: datetime64[ns]


In [116]:
print(df_cleaned['transaction_id'].nunique(), df_cleaned['transaction_id'].shape)


1116389 (1116389,)


In [122]:
df_cleaned[df_cleaned['transaction_id'] == 'TXN_2023_00063013']


,transaction_id,order_date,customer_id,product_id,product_name,category,subcategory,brand,original_price_inr,discount_percent,discounted_price_inr,quantity,subtotal_inr,delivery_charges,final_amount_inr,customer_city,customer_state,customer_tier,customer_spending_tier,customer_age_group,payment_method,delivery_days,delivery_type,is_prime_member,is_festival_sale,festival_name,customer_rating,return_status,order_month,order_year,order_quarter,product_weight_kg,is_prime_eligible,product_rating,corrected_price
0,TXN_2023_00063013,2023-07-23,CUST_2023_00018393,PROD_000454,Vivo Y95 64GB Black,Electronics,Smartphones,Vivo,27340.84,21.57,21442.59,1,21442.59,0.0,21442.59,Kolkata,West Bengal,Metro,Standard,36-45,Cash,3.0,Standard,False,False,No Festival,4.0,Delivered,7,2023,3,0.2,True,3.5,27340.84
